# 🍷 Red Wine Quality Analysis

## 🎯 Goal & Scope

The goal of this analysis is to identify which factors most strongly influence perceived red wine quality. The focus is on developing an explanatory model to understand these relationships, rather than a predictive model aimed at forecasting quality scores.

## 🚰 Data Source
- Kaggle: https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009/data



### **Null Hypothesis** :
There is no linear relationship between wine quality and the predictor variables (alcohol, sulphates, volatile acidity, and density)

---

### 🧠 Interpretation & Insights

Based on the regression output, the coefficients for alcohol, sulphates, and volatile acidity are statistically significant (p < 0.05). This provides strong evidence against the null hypothesis.

**Therefore we reject H₀**

There is a relationship between wine quality and the predictor variables (alcohol, sulphates, volatile acidity)**


- 🍷 **Alcohol, sulphates, and volatile acidity** are the strongest predictors of wine quality.
- The model explains a **statistically significant portion** of variation in ratings, though not all — subjective taste remains a factor.
- **High VIF values** were detected among several chemical variables, indicating **multicollinearity** (some predictors are strongly correlated).
  - This means coefficients should be interpreted with caution — while significant relationships exist, their *independent effects* are less certain.
- Residual plots suggest no major violations of linearity or homoscedasticity, but slight heteroscedasticity was observed at higher fitted values.

### 📈 Model Summary

$$
Quality = 2.57 + 0.31(Alcohol) + 0.68(Sulphates) - 1.24(Volatile\ Acidity)
$$

| Variable | Coefficient | Interpretation |
|:--|:--|:--|
| **Alcohol** | +0.31 | Higher alcohol content generally increases perceived quality |
| **Sulphates** | +0.68 | Wines with higher sulphate levels tend to score higher |
| **Volatile Acidity** | −1.24 | Higher volatile acidity (vinegary aroma) lowers perceived quality |


---






## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings


from ISLP.models import ( ModelSpec as MS, summarize)
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

from plotting_tools import abline, plot_residuals_leverage

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')


In [ ]:
df_raw = pd.read_csv('/Users/guywinfield/PycharmProjects/Data/Red Wine Quality/winequality-red.csv')

In [ ]:
df = df_raw.copy()

In [ ]:
df.head()

## Data Quality Check

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df = df[df.duplicated(keep='last') == False]
df

In [ ]:
df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.describe()

## Exploratory Data Analysis

In [ ]:
fig, ax = plt.subplots(12, 2, figsize=(15, 50))

row = 0
for col in df.columns:
    sns.histplot(data=df, x=col, ax=ax[row,0]).set_title(f"Overall Distribution {col}")
    sns.boxplot(data=df, x='quality', y=col, ax=ax[row,1]).set_title(f"Overall Distribution {col} by quality")
    row += 1

plt.subplots_adjust(hspace=0.3)
plt.show()

At first glance the data doesn't appear to display any worrying outliers with most distributions being either normal or with a right skew.

However we'll take a look at some of the larger outliers for those right skewed distributions as there appear to be a handful of values pretty far away from the mean, we'll take a look at:
- Fixed Acididty
- residual sugar
- chlorides
- sulfur dioxide
- sulphates

We'll have a intuitive check to see if there's anything erroneous and will consider using their empirical rule (68-95-97) to remove any if we're suspicious.


In [ ]:
df_z_scores = df.copy()

for col in df_z_scores.columns:
    df_z_scores[f"z_{col}"] = stats.zscore(df_z_scores[col])

df_z_scores = df_z_scores[df_z_scores[[
    "z_fixed acidity",
    "z_volatile acidity",
    "z_citric acid",
    "z_residual sugar",
    "z_chlorides",
    "z_free sulfur dioxide",
    "z_total sulfur dioxide",
    "z_density",
    "z_pH",
    "z_sulphates",
    "z_alcohol",
    "z_quality"]]
.any(axis=1)]

df_z_scores.describe().T

In [ ]:
df_z_scores[df_z_scores['z_chlorides'] > 10.55]

In [ ]:
df_z_scores[df_z_scores['z_total sulfur dioxide'] > 7.2]

In [ ]:
df_z_scores.sort_values(by=['z_total sulfur dioxide'],ascending=False).head(5)

It looks like we have a couple of candidates for outliers:
- chlorides -> Over 7 standard deviations from the mean
- total sulfur dioxide -> over 10.5 standard deviations from the mean

Let's see their relationship with other features to provide extra evidence of being outliers and potentially erroneous data.

In [ ]:
sns.pairplot(data=df, hue='quality',palette='muted')

### Outliers
We can see the outliers with very high Total Sulfur Dioxide readings. These could well be legitimate wines but as we don't want to skew our data we'll remove those 2 records.

In [ ]:
df.drop([1079, 1081], inplace=True)

## Trends
We can see there are some progressions in colour amongst the scatterplots but these really aren't obvious to see.

Let's add some categorical hue to the data so that we can group our wines better, it'll also mean we can include quality as a feature for compariuson which should make life much easier to see which features have a linear trend with quality.




In [ ]:
df['quality_label'] = pd.qcut(
    df['quality'],
    q=3,
    labels=['low', 'average', 'high']
)

In [ ]:
df['quality_label'].value_counts()

In [ ]:
sns.pairplot(data=df, hue='quality_label', palette='muted')

## Bingo! 👏

We can start to see some linear trends! If we take a look at the last column on the right where we plot quality on the x axis we can see a few trends:
(We'll measure correlations afterwards but this gives us a good starting point on where to look)

- Alcohol: There appears to be a positive correlation
- sulphates: There appears to be a positive correlation
- density: There appears to be a negative correlation
- volatile acidity: There appears to be a negative correlation



In [ ]:
df.iloc[:, :-1].corr()

In [ ]:
plt.figure(figsize=(12, 8), dpi=80)

sns.heatmap(data=df.iloc[:, :-1].corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
feature_cols = ['quality','alcohol','sulphates','volatile acidity','density']

df_stats = df[feature_cols]
df_stats.corr()

## Simple Linear Regression

Now we know what features we want to work with let's dig deeper into the strength of these relationships starting with a Simple Linear Regression where we'll compare our most highly correlated predictor/inpout variable with our outcomes variable:
- X: alcohol
- Y: quality

**Null Hypothesis: The level of alcohol have no perceived impact on quality of wine**

Confidence Threshold (alpha): 0.05

In [ ]:
X1 = pd.DataFrame({'intercept': np.ones(df.shape [0]),'alcohol': df['alcohol']})
X1.head()

In [ ]:
y1 = df['quality']
model1 = sm.OLS(y1, X1)
results1 = model1.fit()

summarize(results1)

### What is this telling us?
#### There’s a strong positive relationship between alcohol content and wine quality


- The p-values (P>|t|) are 0.0, meaning the coefficients are statistically significant as they are below the 0.05 threshold.
- The t-values are very large, which confirms that the relationship between alcohol and quality is not random.
- On average, wines with higher alcohol content are rated as higher quality.
- For each 1% increase in alcohol content, wine quality increases by roughly 0.36 points, holding everything else constant

**On this basis with P>0.05, we can reject the null hypothesis** that the level of alcohol has no perceived impact on wine quality.

In [ ]:
ax = df.plot.scatter('alcohol', 'quality')
abline(ax ,results1.params [0],results1.params [1],'r--',linewidth =1)

#### Sheeeesh, that's not pretty.... 🫣

Although there is a statistically significant relationship it's very coarse to create a linear relationship with a discrete outcome variable. An alternative could be to use a binary outcome of 'good' or 'bad' wine where we could apply a logistic regression.

For now, let's check the homodescacity and Leverage Values


#### (Left Chart) Homodescacity -> Checking Residuals on fitted values

The residuals are values which fall either above or below our fitted line. We want the spread of the residuals to be evenly spread across the 0 so that our model isn't losing accuracy as it increase in X (We'd see this as a fanning out of residual distance further along x we go like a funnel).

This chart suggests our model’s residuals are structured rather than random likely due to the discrete outcome variable (quality score).

It’s not necessarily a model violation, but it means the linear model might not fully capture all patterns in the data. 🫸



#### (Right Chart) Leverage (Hat) Values Plot (Right Plot) ->
Leverage shows how far an observation’s predictor values are from the mean of all predictor values, high leverage points can disproportionately influence the model’s fitted coefficients.

Most hat values are very small (below 0.005) but a few points (around indices 300–500) have slightly higher leverage (up to ~0.015), but still not extreme.

This means the model has no major outliers ✅

In [ ]:
plot_residuals_leverage(results1)

### Let's  make a cleaner Simple Regression

Let's re-try a simple linear regression with another 2 variables to have a cleaner example. In our correlation heatmap we saw a strong correlation between `density` and `fixed acidity`

**Null Hypothesis: The level of density has no perceived impact on the fixed acidity of wine**

Confidence Threshold (alpha): 0.05


In [ ]:
X2 = pd.DataFrame({'intercept': np.ones(df.shape [0]),'fixed acidity': df['fixed acidity']})
y2 = df['density']

model2 = sm.OLS(y2, X2)
results2 = model2.fit()

summarize(results2)

**On this basis with P>0.05, we can reject the null hypothesis** that desnity has no perceived impact on the fixed acidity of wine.

In [ ]:
ax = sns.scatterplot(data=df, x='fixed acidity', y='density')
abline(ax ,results2.params [0],results2.params [1],'r--',linewidth =1)

#### (Left Chart) Homodescacity -> Checking Residuals on fitted values

The residuals are values which fall either above or below our fitted line. We want the spread of the residuals to be evenly spread across the 0 so that our model isn't losing accuracy as it increase in X (We'd see this as a fanning out of residual distance further along x we go like a funnel).

In this case our residuals seem to be consistently spread above and below 0.

This means the model is more trustworthy ✅

#### (Right Chart) Leverage (Hat) Values Plot (Right Plot) ->
Leverage shows how far an observation’s predictor values are from the mean of all predictor values, high leverage points can disproportionately influence the model’s fitted coefficients.

Most hat values are very small (below 0.005) but a few points (around indices 400–600) have slightly higher leverage (up to ~0.015), but still not extreme.

This means the model has no major outliers ✅

In [ ]:
plot_residuals_leverage(results2)

Now let's make a prediction to see how much density we could expect from a wine with 12 fixed acidity.

In [ ]:
while True:
    density_input = input("Density Input (0.994 -> 1.003): ")
    try:
        density_input = float(density_input)
        break
    except ValueError:
        print("⚠️ Please enter a float number.")

design = MS(['density'])
design.fit(df)
X = design.transform(df)

# 2) Fit OLS model
y = df['fixed acidity']
results2 = sm.OLS(y, X).fit()

# 3) Predict for density = 0.98
pred_df = pd.DataFrame({'density': [density_input]})
newX = design.transform(pred_df)

pred = results2.get_prediction(newX)
pred_mean = pred.predicted_mean[0]
pred_ci = pred.conf_int(alpha =0.05)

print(f"Given {density_input} density \nPredicted fixed acidity: {round(pred_mean,2)} \nConfidence Intervals: {pred_ci}")

## Multiple Linear Regression

Let's see if we can create a more detailed explanation of the relationship with wine quality when including multiple features.

As a reminder we'll use these inputs for our model as they correlate best with quality.
- x: alcohol, sulphates, volatile acidity, density
- Y: quality

We'll check to see whether we run into any multicollinearity afterwards.

**Null Hypothesis** : There is no linear relationship between wine quality and the predictor variables (alcohol, sulphates, volatile acidity, and density).


In [ ]:
df_stats.corr()

Based on our results it looks like density isn't a useful feature in this model therefore we'll remove it

In [ ]:
x3 = MS(['alcohol','sulphates','volatile acidity','density']).fit_transform(df)
Y3 = df['quality']

model3 = sm.OLS(Y3, x3)
results3 = model3.fit()
summarize(results3)

Given that there's no statistically significant realtionship with density let's remove it

In [ ]:
x3 = MS(['alcohol','sulphates','volatile acidity']).fit_transform(df)
Y3 = df['quality']

model3 = sm.OLS(Y3, x3)
results3 = model3.fit()
summarize(results3)

## 🍷 Findings Explained
The relationship between quality and alcohol, sulphates and volatile acidity are statistically significant. If all predictors (alcohol, sulphates, volatile acidity) were 0, the predicted wine quality score would be 2.57.

- Alcohol: Holding other variables constant, a **1-unit increase in alcohol** is associated with a **+0.31 increase in predicted wine quality**. *Wines with higher alcohol content tend to receive higher quality ratings.*

- Sulphates: Each **1-unit increase in sulphates** increases predicted quality by **0.68 points**, keeping other variables fixed. *Sulphates, which help with preservation and flavour stability, are linked to better quality.*

- Volatile Acidity: A **1-unit increase in volatile acidity** decreases predicted wine quality by **1.24 points**, assuming other factors stay constant. *Higher volatile acidity (vinegary aroma or taste) lowers perceived quality.*

Putting it together, our fitted regression equation looks like:

$$
Quality = 2.57 + 0.31(Alcohol) + 0.68(Sulphates) - 1.24(Volatile\ Acidity)
$$


## ✅ Summary Table

| Predictor | Effect Direction | Strength | Significance |
|------------|------------------|-----------|---------------|
| Alcohol | 🔼 Positive | Strong | ✅ Significant |
| Sulphates | 🔼 Positive | Moderate | ✅ Significant |
| Volatile Acidity | 🔽 Negative | Strong | ✅ Significant |


# Outcome
Based on the regression output, the coefficients for alcohol, sulphates, and volatile acidity are statistically significant (p < 0.05).
This provides strong evidence against the null hypothesis.

**Therefore, we reject H₀.**

In [ ]:
plot_residuals_leverage(results3)

In [ ]:
features = ['alcohol','sulphates','volatile acidity','quality']

df_stats = df[features]

## 🔎 VIF Results

When you see a very high VIF (like above 10), it basically means that some of your predictor variables (like alcohol, sulphates, and volatile acidity) are kind of "overlapping" in the information they bring to the table. In other words, they're a bit too similar or too closely related to each other.

- Statistical significance might still be there: You might still find that alcohol, sulphates, or acidity each shows up as "statistically significant," meaning they each seem related to quality on their own.

- But prediction stability is the issue: When you have high VIFs, it means the model’s coefficients might be quite unstable. Small changes in the data can make those coefficients jump around because the model can’t easily separate out which variable is doing what.

#### Therefore this means although we shouldn't base a prediction on this model we can use it to understand the relationships for quality in wine

In [ ]:
# Calculate VIF for each numerical feature
vif_data = pd.DataFrame()
vif_data["feature"] = df_stats.columns

# Calculate VIF and round to 4 decimal places
vif_data["VIF"] = [round(variance_inflation_factor(df_stats.values, i), 4) for i in range(df_stats.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)

# Display the VIF DataFrame
print(vif_data)

Ok let's try another model where our y variable isn't a discrete variable.

In [ ]:
plt.figure(figsize=(12, 8), dpi=80)

sns.heatmap(data=df.iloc[:, :-1].corr(), annot=True, cmap='coolwarm')
plt.show()

Let's have a look to see what the most highly correlated features are. Citric Acid seems to have the strongest aggregate relationships with other feature so let's see if we can create a model to predict it.

Based on the heatmap above it looks we'll use the below inputs:
- x: fixed acidity, volatile acidity, density, pH, sulphates
- Y: citric acid

In [ ]:
df.iloc[:, :-1].corr().abs().sum().sort_values()

It looks like we risk some multicollinearity between the different acidity based features.

In [ ]:
feature_cols = ['fixed acidity', 'volatile acidity', 'density', 'pH', 'sulphates', 'citric acid']

df_stats = df[feature_cols]
df_stats.corr()

In [ ]:
x4 = MS(['fixed acidity', 'volatile acidity', 'pH', 'sulphates']).fit_transform(df)
Y4 = df['citric acid']

model4 = sm.OLS(Y4, x4)
results4 = model4.fit()
summarize(results4)

In [ ]:
plot_residuals_leverage(results4)

It looks like we're going to run into multicollinearity issues here as some of the acid based features may not be independent of each other.

To check how much of a multicollinearity there is let's use the Variance Inflation Factor (VIF). It gives a numerical value that indicates how much the variance of a regression coefficient is inflated due to multicollinearity.

In short:
- VIF > 5: indicates moderate multicollinearity
- VIF > 10: indicates severe multicollinearity.





In [ ]:
# Calculate VIF for each numerical feature
vif_data = pd.DataFrame()
vif_data["feature"] = df_stats.columns

# Calculate VIF and round to 4 decimal places
vif_data["VIF"] = [round(variance_inflation_factor(df_stats.values, i), 4) for i in range(df_stats.shape[1])]

# Sort VIF values in descending order
vif_data = vif_data.sort_values(by="VIF", ascending=False)

# Display the VIF DataFrame
print(vif_data)

This shows us that there's far too much multicollinearity between our features to make a reliable prediction as the coefficients for these variables are too dependant and nosiy.

However this does tell is that there's a huge amount of acidic crossover when looking at wines!